In [761]:
# builtin deps
import os
import ssl
import time
import typing as t
from pathlib import Path
from collections import defaultdict
from urllib.request import urlopen, Request

# third parties deps
import pandas as pd
import numpy as np

# Load dataset

## Load consumption data from csv file to dataframe

In [762]:
os.makedirs("./clean_datasets", exist_ok = True)

# path to datasets for visualization
DATASETS_DIR = Path("./datasets")
CLEAN_DATASETS_DIR = Path("./clean_datasets")

# weather file
WEATHER_CSV = DATASETS_DIR / "weather_data.csv"

In [763]:
def load_csvs(src_dir: Path) -> t.Tuple[t.Dict[str, pd.DataFrame], t.List[str]]:
    # get all datasets name in a new list
    dataset_csvs = list(src_dir.iterdir())

    # initiate empty defaultdict
    dfs = defaultdict(lambda : pd.DataFrame())

    # iterate through the list of csvs and record only the filename (not extension)
    # to the default dictionary dfs
    for csv in dataset_csvs:
        dfs[csv.name.split('.')[0]] = pd.read_csv(csv)

    # records filenames to the list
    csv_names = list(dfs.keys())
    
    return dfs, csv_names

In [766]:
dfs, csv_names = load_csvs(DATASETS_DIR)

In [767]:
print("All file names\n---------- ", *sorted(csv_names), sep="\n")

All file names
---------- 
meter1_phase1
meter1_phase2
meter1_phase3
meter2_phase1
meter2_phase2
meter2_phase3
people
weather_data


## Load weather data

The best website for scraping weather data at Chiang Mai that I have found is [scraping path] in [กรมอุตุภาคเหนือ].
  

Scraping url example:  
http://www.aws-observation.tmd.go.th/web/aws/popup_timeseries_tbl.asp?gval=3&id=1&sdate=20201210230100&timewidth=2h&celltime=2  
Format:  
http://www.aws-observation.tmd.go.th/web/aws/popup_timeseries_tbl.asp?gval=3&id=1&sdate={YYYYMMDDHHmmss}&timewidth={numhour}h&celltime={num_hour_to_be_displayed}  
*Note that, the result's time in the table is UTC time, we need to convert to GMT+7 time*
  
**Mock user agent to access some strictly crawler prevention website**  
For mock user agent [mock user] 
  
**Create SSL certification to access https website**  
For creating ssl certification in macos python to connect with https [ssl cert]. 
  
**Another resources for TMD data**  
data from TMD (not free haha) [กรมอุตุ]  
Daily TMD data [accuweather] and [weather.com]
  
[accuweather]: https://www.accuweather.com/th/th/mueang-chiang-mai/317505/january-weather/317505?year=2020  
[weather.com]: https://weather.com/weather/monthly/l/9c4f2b7a834c8555f56c1a4d1f4d4630a84169eda9a08a0c4b947c3de75d7426  
[กรมอุตุ]: https://www.tmd.go.th/services/services.php  
[กรมอุตุภาคเหนือ]: http://www.cmmet.tmd.go.th/#  
[scraping path]: http://www.aws-observation.tmd.go.th/web/weather/weather_calendar.asp  
[mock user]: https://medium.com/@speedforcerun/python-crawler-http-error-403-forbidden-1623ae9ba0f#:~:text=It%20possibly%20due%20to%20the,of%20your%20fake%20browser%20visit.  
[ssl cert]: https://stackoverflow.com/questions/50236117/scraping-ssl-certificate-verify-failed-error-for-http-en-wikipedia-org


In [108]:
URL = "http://www.aws-observation.tmd.go.th/web/aws/popup_timeseries_tbl.asp?gval=3&id=1&sdate=20201210230100&timewidth=2h&celltime=2"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"
}

# new column names for weather df
NEW_COL_NAMES = ['time', 'prec1m_mm', 'prec15m_mm', 'prec30m_mm', 'prec1h_mm',
                 'prec2h_mm', 'prec3h_mm', 'prec6h_mm', 'prec12h_mm', 'prec24h_mm',
                 'wd_deg', 'ws_km-h', 'temp_c', 'humi_perc', 'prec_mm', 'pres_hPa', 'vis_m', 'weather']


In [109]:
"""
Extract transfrom load function
"""
def extract(year: str, month: str, date: str, 
                   hour: str, minute: str = "01", sec: str = "00", 
                   timewidth: str = "1h", celltime: str = "1") -> pd.DataFrame:
    # define url
    datetime = year + month + date + hour + minute + sec
    url = f"http://www.aws-observation.tmd.go.th/web/aws/\
popup_timeseries_tbl.asp?gval=3&id=1&sdate={datetime}&timewidth={timewidth}&celltime={celltime}"

    # GET request with header and retrieve html from website
    req = Request(url=url, headers=HEADERS) 
    html = urlopen(req).read()
    
    # since html website return html's table format, 
    # we can use read_html to convert html to dataframe directly
    weather_df = pd.read_html(html)[0]
    
    return weather_df
    

def transform(df: pd.DataFrame, year: str, month: str, date: str) -> pd.DataFrame:
    
    target = df.copy()
    
    # change weather columns name
    target.columns = NEW_COL_NAMES
    
    # convert 24:00 to 00:00 (to conform to pandas format) and append its date
    target["time"].loc[target.copy()["time"] == "24:00"] = "00:00"
    target["datetime"] = pd.to_datetime("-".join([year, month, date]) + " " + target["time"])
    
    # convert UTC to GMT+7
    target["datetime"] = target["datetime"] + pd.to_timedelta(7, "h")
    target = target.drop(columns=['time'])
    
    return target


def load(df: pd.DataFrame, file: Path):
    '''Append to the existing csv'''
    if file.exists():
        df.to_csv(file, mode="a", header=False, index=False)
    else:
        df.to_csv(file, index=False)    
    
def etl(year: str, month: str, date: str, 
                   hour: str, minute: str = '01', sec: str = '00', 
                   timewidth: str = 1, celltime: str = 1):

    weather_df = extract(year, month, date, hour, minute, sec, timewidth, celltime)
    weather_tf_df = transform(weather_df, year, month, date)
    load(weather_tf_df, WEATHER_CSV)

In [110]:
"""
iterate each hour and apply etl to query weather data
"""
def get_daily_weather_data_loop_hourly(from_date, to_date):
    
    assert from_date < to_date, "from_date must less than or equal to_date!"
    
    # adjust the starting time to be fetched at 17:00 UTC time
    cur_date = from_date + pd.to_timedelta(-7, "h")
    num_hours = int((to_date - from_date).days*24)
    
    for i in range(1, num_hours+1):
        
        # ETL weather data 1 hour per call
        etl(f"{cur_date.year}",f"{cur_date.month:02d}", f"{cur_date.day:02d}", 
            hour = f"{cur_date.hour:02d}", timewidth="1h", celltime="1")
        
        if i % 24 == 0: print(f"Loaded weather data on: {cur_date}")
            
        cur_date = from_date + pd.to_timedelta(i, "h")
            
        # sleep 2 second for every request to prevent rate limiting
        # 30 request per minute
        time.sleep(2)

### Extract transform and load weather data from API to csv file  
*Shouldn't be run carelessly*

In [111]:
# from_date = pd.to_datetime('2020-12-03')
# to_date = pd.to_datetime('2021-01-13')

# get_daily_weather_data_loop_hourly(from_date, to_date)

### Load weather data from csv file to dataframe

In [768]:
dfs['weather'] = pd.read_csv(
    WEATHER_CSV, 
    usecols=['datetime', 'temp_c', 'humi_perc'], 
    dtype={
        'temp_c':float,
        'humi_parc': float
    },
    na_values='-' # NA values of the weather data is '-'
)

dfs['weather'] = dfs['weather'].rename(columns={'datetime':'timestamp'})
csv_names = list(dfs.keys())

In [769]:
csv_names

['meter1_phase1',
 'meter1_phase2',
 'meter1_phase3',
 'people',
 'weather_data',
 'meter2_phase1',
 'meter2_phase3',
 'meter2_phase2',
 'weather']

# Observe completeness of the data

## Info

In [770]:
for csv in csv_names:
    print(csv, end='\n*==================*\n')
    print(dfs[csv].info(), end="\n\n")

meter1_phase1
*==================*
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72446 entries, 0 to 72445
Data columns (total 10 columns):
timestamp                  72446 non-null object
energy_reactive            72446 non-null float64
current                    72446 non-null float64
power_reactive             72446 non-null float64
energy_reactive_to_grid    72446 non-null float64
voltage                    72446 non-null float64
energy                     72446 non-null float64
energy_to_grid             72446 non-null int64
power                      72446 non-null float64
power_factor               72446 non-null float64
dtypes: float64(8), int64(1), object(1)
memory usage: 5.5+ MB
None

meter1_phase2
*==================*
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3661 entries, 0 to 3660
Data columns (total 6 columns):
timestamp         3661 non-null object
current           3661 non-null float64
power_reactive    3661 non-null float64
voltage           3661 non-null

## describe the data

In [771]:
for csv in csv_names:
    print(csv, end='\n*==================*\n')
    print(dfs[csv].describe(), end="\n\n")

meter1_phase1
*==================*
       energy_reactive       current  power_reactive  energy_reactive_to_grid  \
count     72446.000000  72446.000000    72446.000000             72446.000000   
mean      37369.768315     16.497044       -1.351647              7364.072086   
std          74.605142      8.950949        0.781349               279.621576   
min       37268.031250      0.227977       -4.071399              6832.749023   
25%       37288.490234     10.244700       -1.852476              7086.360840   
50%       37375.402344     15.958839       -1.458420              7395.241455   
75%       37431.357422     21.858104       -1.085153              7645.033203   
max       37485.367188     57.309692        4.133070              7696.834961   

            voltage         energy  energy_to_grid         power  power_factor  
count  72446.000000   72446.000000         72446.0  72446.000000  72446.000000  
mean     404.743728  143093.745012             0.0      3.545585      0.9

## Show head of the data frame

In [772]:
for csv in csv_names:
    print(csv, end='\n*==================*\n')
    print(dfs[csv].head())

meter1_phase1
*==================*
             timestamp  energy_reactive    current  power_reactive  \
0  2020-12-19 00:00:27      37268.03125  24.109783       -1.756851   
1  2020-12-19 00:00:57      37268.03125  24.554781       -1.800468   
2  2020-12-19 00:01:27      37268.03125  16.898428       -1.738132   
3  2020-12-19 00:01:57      37268.03125  16.988962       -1.746844   
4  2020-12-19 00:02:28      37268.03125  17.363531       -1.746971   

   energy_reactive_to_grid     voltage         energy  energy_to_grid  \
0              6832.749023  403.224792  140042.203125               0   
1              6832.780762  402.509125  140042.296875               0   
2              6832.790039  402.380371  140042.406250               0   
3              6832.799805  402.881500  140042.500000               0   
4              6832.809082  402.580475  140042.593750               0   

      power  power_factor  
0  5.314347      0.949463  
1  5.400183      0.948662  
2  3.506593      0.89

## Number of entries

In [773]:
for csv in csv_names:
    print(f"{csv}\n", "number of entries:", len(dfs[csv]), end="\n\n")

meter1_phase1
 number of entries: 72446

meter1_phase2
 number of entries: 3661

meter1_phase3
 number of entries: 68595

people
 number of entries: 3629

weather_data
 number of entries: 59100

meter2_phase1
 number of entries: 73191

meter2_phase3
 number of entries: 26600

meter2_phase2
 number of entries: 73186

weather
 number of entries: 59100



## range of the data

In [774]:
def print_range_of_timeseries(dfs):
    for key in dfs:
        print(key)
        field = 'timestamp' if 'timestamp' in dfs[key].columns else 'datetime'
        fr, to = dfs[key][field].iloc[0], dfs[key][field].iloc[-1]
    
        print(f'from: {fr} - to: {to}', end='\n\n')

In [775]:
print_range_of_timeseries(dfs)

meter1_phase1
from: 2020-12-19 00:00:27 - to: 2021-01-13 17:38:20

meter1_phase2
from: 2020-12-19 00:00:27 - to: 2020-12-20 06:30:57

meter1_phase3
from: 2020-12-19 00:00:27 - to: 2021-01-13 17:42:20

people
from: 2020-12-15 22:01:44 - to: 2021-01-11 10:25:02

weather_data
from: 3/12/2020 07:01 - to: 13/1/2021 08:00

meter2_phase1
from: 2020-12-19 00:00:28 - to: 2021-01-13 17:56:51

meter2_phase3
from: 2020-12-19 00:00:28 - to: 2020-12-27 02:31:58

meter2_phase2
from: 2020-12-19 00:00:28 - to: 2021-01-13 18:00:21

weather
from: 3/12/2020 07:01 - to: 13/1/2021 08:00



*Note that, the string date format of the weather is different from meter and people format*

## Is Nan Checking

In [776]:
for csv in csv_names:
    print(csv, end=f"\n========\n")
    print(dfs[csv].isna().any(), end="\n\n")

meter1_phase1
timestamp                  False
energy_reactive            False
current                    False
power_reactive             False
energy_reactive_to_grid    False
voltage                    False
energy                     False
energy_to_grid             False
power                      False
power_factor               False
dtype: bool

meter1_phase2
timestamp         False
current           False
power_reactive    False
voltage           False
power             False
power_factor      False
dtype: bool

meter1_phase3
timestamp         False
current           False
power_reactive    False
voltage           False
power             False
power_factor      False
dtype: bool

people
datetime    False
zone1       False
zone2       False
zone3       False
zone4       False
dtype: bool

weather_data
prec1m_mm     False
prec15m_mm    False
prec30m_mm    False
prec1h_mm     False
prec2h_mm     False
prec3h_mm     False
prec6h_mm     False
prec12h_mm    False
prec24h_mm    Fals

**There are NaN values on weather data**

In [777]:
dfs['weather'].isna().sum()/len(dfs['weather'])*100

temp_c       0.00846
humi_perc    0.00846
timestamp    0.00000
dtype: float64

*Only 0.00846% of the data is missing* 

In [778]:
dfs['weather'].loc[dfs['weather'].isna()['temp_c']]

,temp_c,humi_perc,timestamp
8917,NaN,NaN,9/12/2020 11:38
8918,NaN,NaN,9/12/2020 11:39
8919,NaN,NaN,9/12/2020 11:40
8920,NaN,NaN,9/12/2020 11:41
8921,NaN,NaN,9/12/2020 11:42


In [779]:
selected_csv_names = ['meter1_phase1', 'meter2_phase1', 'weather', 'people']

# Transform and clean the data

## Fill weather NA value with prev valid value

In [780]:
# fill NA value with previous valid observation
dfs['weather'].fillna(method='ffill', inplace=True)

In [781]:
dfs['weather'].isna().any()

temp_c       False
humi_perc    False
timestamp    False
dtype: bool

## tranfrom string timestamp to datetime timestamp

In [782]:
# convert string datetime to datetime type and append to the dataframe
for name in selected_csv_names:
    if name == 'weather':
        # format of the weather is different from 
        dfs[name]['_datetime'] = pd.to_datetime(dfs[name].timestamp, dayfirst=True)
    elif name != 'people':
        dfs[name]['_datetime'] = pd.to_datetime(dfs[name].timestamp)
    else:
        dfs[name]['_datetime'] = pd.to_datetime(dfs[name].datetime)

In [783]:
def group_multilevel_ind(df: pd.DataFrame, interval: str) -> pd.DataFrame:
    
    target = df.copy()
    
    if interval == 'hourly':
        target.index = pd.to_datetime(pd.DataFrame({
            'year':df.index.get_level_values(0),
            'month':df.index.get_level_values(1),
            'day':df.index.get_level_values(2),
            'hour':df.index.get_level_values(3),
        }))
    else:
        target.index = pd.to_datetime(pd.DataFrame({
            'year':df.index.get_level_values(0),
            'month':df.index.get_level_values(1),
            'day':df.index.get_level_values(2),
        }))        

    return target

In [784]:
def extract_to_hourly_data(name: str, dfs: t.Dict[str, pd.DataFrame]):
    return dfs[name]
    

def transform_to_hourly_data(
    df: pd.DataFrame,
    fields: t.List[str]):
        
    # group by year month day and hour,
    # then average the value if each grouped as a representation of that group
    output = pd.DataFrame(
        df[fields].groupby([
            df._datetime.dt.year, 
            df._datetime.dt.month, 
            df._datetime.dt.day, 
            df._datetime.dt.hour]).mean())
        
    # group multilevel index to datetime
    output = group_multilevel_ind(output, 'hourly')
        
    return output
        

def load_hourly_data(df: pd.DataFrame, name):
    '''Load cleaned data to clean_datasets dir'''
    df.to_csv(CLEAN_DATASETS_DIR / (name+'.csv'))
    

def etl_hourly_data(name: str, fields: str, dfs: t.Dict[str, pd.DataFrame]):
    df = extract_to_hourly_data(name, dfs)
    output = transform_to_hourly_data(df, fields)
    load_hourly_data(output, name)

## Extract raw meter1 data, transform to 1 hour interval data and load to clean_datasets dir

In [785]:
etl_hourly_data('meter1_phase1', ['power'], dfs)

## Extract raw meter2 data, transform to 1 hour interval data and load to clean_datasets dir

In [786]:
etl_hourly_data('meter2_phase1', ['power'], dfs)

## Extract raw weather data, transform  1 hour interval and load to clean_datasets dir

In [787]:
etl_hourly_data('weather', ['temp_c'], dfs)

## Extract raw people data transform to 1 hour interval and load to clean_datasets dir

In [788]:
etl_hourly_data('people', ['zone1', 'zone2', 'zone3', 'zone4'], dfs)